In [12]:
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch
from typing import List, Dict, Union
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [21]:
DATA_DIR: str = "datasets"
IMG_SIZE: int = 12 # 256
NUM_FRAMES_PER_VIDEO: int = 16
targets: Dict[str, int] = {"1": 0, "2": 0, "3": 1, "4": 1, "5": 2, "6": 2, \
    "7": 3, "8": 3, "HR_1": 0, "HR_2": 1, "HR_3": 2, "HR_4": 3}

def video2frames(video_path: str, resize: Union[int, int] = (IMG_SIZE, IMG_SIZE)) -> np.array:
    cap = cv2.VideoCapture(video_path)
    frames: list = []
    is_there_frame: bool = True
    num_total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)    
    resampling_rate: int = int(num_total_frames / NUM_FRAMES_PER_VIDEO)
    idf: int = 0
    while is_there_frame and len(frames) < NUM_FRAMES_PER_VIDEO:
        idf += 1
        is_there_frame, frame = cap.read()
        if idf % resampling_rate == 0:
            # grayscale
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # resize
            frames.append(frame)
    assert len(frames)==NUM_FRAMES_PER_VIDEO
    return np.array(frames)

def load_casia_dataset() -> Union[list, list]:
    try:
        num_folders: int = 1
        dataset_x: list = []
        dataset_y: list = []
        with glob.glob(f'{DATA_DIR}/**/*.avi', recursive=True) as files:
            files = glob.glob(f'{DATA_DIR}/**/*.avi', recursive=True)
            for f in files:
                frames: np.array = video2frames(f)
                dataset_x = [*dataset_x, *frames]
                dataset_x = np.array(dataset_x)
                vector_path: List[str] = f.split("/")
                tmp = [targets[vector_path[-1][:-4]]]*NUM_FRAMES_PER_VIDEO
                dataset_y = [*dataset_y, *tmp]
    except LookupError: # IndexError, KeyError
        print("index from video name not found in map targets")
    return dataset_x, dataset_y

In [22]:
types: List[str] = set(targets.values())
batch_size: int = 32

{0, 1, 2, 3}

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=DATA_DIR+"/train",
                                        batch_size=batchsize,
                                        shuffle=True)


In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)
for e in resnet18.parameters():
   e.requires_grad = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet18.fc = nn.Linear(in_features=512, out_features=len(types), bias=True)
optimizer = optim.SGD(resnet18.parameters(), lr=1e-4, weight_decay=0.005, momentum=0.9)
resnet18.to(device)

In [ ]:
train_loss_resnet18, val_loss_resnet18 = train(resnet18, optimizer, loss_fn, 50)